In [19]:
!apt-get update
!apt-get install -y assimp-utils libassimp-dev

!pip install open3d plotly
!pip install pyassimp

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,047 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,741 kB]
Get:14 

In [20]:
import copy
import pyassimp
import numpy as np
import pandas as pd
import open3d as o3d
import plotly.graph_objects as go

## Plotting function

In [5]:
def plotMesh(modelMesh):
  mesh = copy.deepcopy(modelMesh);
  # Compute normals
  mesh.compute_vertex_normals()
  mesh.compute_triangle_normals()

  # Turn into np array
  vertices = np.asarray(mesh.vertices)
  triangles = np.asarray(mesh.triangles)

  # plotly
  fig = go.Figure(data=[
      go.Mesh3d(
          x=vertices[:, 0],
          y=vertices[:, 1],
          z=vertices[:, 2],
          i=triangles[:, 0],
          j=triangles[:, 1],
          k=triangles[:, 2],
          color='lightblue',
          opacity=1.0
      )
  ],
  layout=dict(
          scene=dict(
              xaxis=dict(visible=False),
              yaxis=dict(visible=False),
              zaxis=dict(visible=False)
          )
      )

  )

  fig.show()

## Load `.OBJ` `.GLTF` `.STL` models



In [8]:
bunny = o3d.io.read_triangle_mesh("bunny.obj")
bear = o3d.io.read_triangle_mesh("bear.gltf")
teapot = o3d.io.read_triangle_mesh("teapot.stl")

## Comparison between models

In [11]:
def mesh_info(mesh):
    # Some formats may not include normals
    if len(mesh.vertex_normals) == 0:
        mesh.compute_vertex_normals()

    return {
        "Vertices": len(mesh.vertices),
        "Faces": len(mesh.triangles),
        "Normals": len(mesh.vertex_normals),
    }

data = {
    "Bunny (.obj)": mesh_info(bunny),
    "Bear (.gltf)": mesh_info(bear),
    "Teapot (.stl)": mesh_info(teapot),
}

df = pd.DataFrame(data)
print(df)

          Bunny (.obj)  Bear (.gltf)  Teapot (.stl)
Vertices          2503          6857          23324
Faces             4968         12126           9438
Normals           2503          6857          23324


In [12]:
plotMesh(bunny)

In [13]:
plotMesh(bear)

In [14]:
plotMesh(teapot)

## Converting to other formats

In [21]:
import pyassimp

# OBJ -> STL
with pyassimp.load("bunny.obj") as scene:
    pyassimp.export(scene, "bunny.stl", file_type="stl")

# GLTF -> OBJ
with pyassimp.load("bear.gltf") as scene:
    pyassimp.export(scene, "bear.obj", file_type="obj")

# STL -> GLTF
with pyassimp.load("teapot.stl") as scene:
    pyassimp.export(scene, "teapot.gltf", file_type="gltf2")